# zhsegment: default program

In [2]:
from default import *

## Run the default solution on dev

In [4]:
Pw_unigram = Pdist_unigram(data=datafile("data/count_1w.txt"), missingfn=avoid_long_words)
Pw_bigram = Pdist_bigram(data=datafile("data/count_2w.txt"))
Pw_trigram = Pdist_trigram(data=datafile("data/count_3w.txt"))
segmenter = Segment(Pw_unigram, Pw_bigram, Pw_trigram, n_gram=3, smooth_prob=True) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白国良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。


## Evaluate the default output

In [5]:
from zhsegment_check import fscore
with open('data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.5f}".format(tally), file=sys.stderr)


score: 0.92561


## Documentation

Write some beautiful documentation of your program here.

### We have implemented 5 models.
1. `UNIGRAM` is a plain unigram model.
<br/>
<br/>
2. `BIGRAM WITH STUPID-BACKOFF` uses stupid backoff which is it backsoff to unigram from bigrams if no bigram is found. It multiplies each probability by `delta=0.4` at each back off step. The function `bigram_prob_stupid_backoff` calculates this probabilty.
<br/>
<br/>
3. `BIGRAM WITH JM-SMOOTHING` uses lambda to interpolate the bigram and unigram probabilities such that (lambda)(bigram_prob) * (1-lambda)(unigram_prob). The function `bigram_prob_jm_smoothing` calculates this probability.
<br/>
<br/>
4. `TRIGRAM WITH STUPID-BACKOFF` uses stupid backoff from tigram to bigram to unigram. At each step we multiply a additional amount of `0.4`. The function `trigram_prob_stupid_backoff` calculates this probability.
<br/>
<br/>
5. `TRIGRAM WITH INTERPOLATION-SMOOTHING` uses three lambda values `lambda_1`, `lambda_2` and `lambda_3` to linearly interpolate the trigram, bigram and unigram probabilities respectively such that `lambda_1 + lambda_2 + lambda_3 = 1`. The function `trigram_prob_interpolation` calculates this probability.

### We found the best results with the linearly interploated trigram model `TRIGRAM WITH INTERPOLATION-SMOOTHING`. So we are submitting that. We have set the `default.py` to run that model by setting `model_version` by default to it.

#### NOTE: For the trigrams model we used the `data/train.txt` file to extract the trigram counts. The function `extract_trigrams()` does it. The code is set in such a way that on running `default.py` it checks whether the trigram count file `data/count_3w.txt` exists. If not it extracts the trigrams. We have added the `count_3w.txt` to `answers/` if the trigram counts fail to be extracted.

In [ ]:
Following are the Fscores on dev and val dataset for all of these models

UNIGRAM:                             | dev: 0.92519
BIGRAM WITH STUPID-BACKOFF           | dev: 0.92327
BIGRAM WITH JM-SMOOTHING             | dev: 0.92492
TRIGRAM WITH STUPID-BACKOFF          | dev: 0.92389
TRIGRAM WITH INTERPOLATION-SMOOTHING | dev: 0.92561

## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?

### Through a greedy search we found optimal lambda values

#### Greed search for jm_lambda for BIGRAM MODEL WITH JM-SMOOTHING: Best lambda value = 0.77

In [11]:
# Greedy search
from default import *
from zhsegment_check import fscore

Pw_bigram = Pdist_bigram(data=datafile("data/count_2w.txt"))
Pw_trigram = Pdist_trigram(data=datafile("data/count_3w.txt"))
Pw_unigram = Pdist_unigram(data=datafile("data/count_1w.txt"), missingfn=avoid_long_words)

jm_lambdas = [i*0.01 for i in range(1,99)]

best_f1 = 0
best_lambda = 0

for jm_lambda in jm_lambdas:
    segmenter = Segment(Pw_unigram, Pw_bigram, Pw_trigram, n_gram=2, smooth_prob=True, jm_lambda=jm_lambda) # note that the default solution for this homework ignores the unigram counts
    output_full = []
    with open("data/input/dev.txt") as f:
        for line in f:
            output = " ".join(segmenter.segment(line.strip()))
            output_full.append(output)

    with open('data/reference/dev.out', 'r') as refh:
        ref_data = [str(x).strip() for x in refh.read().splitlines()]
        tally = fscore(ref_data, output_full)
        if best_f1 < tally:
            best_f1 = tally
            best_lambda = jm_lambda
            print("BEST: ", best_f1, " ", best_lambda)

BEST:  0.9247698504027618   0.01
BEST:  0.9247870651075503   0.75
BEST:  0.9249205890846087   0.77


#### Greed search for lambdas for TRIGRAM MODEL WITH INTERPOLATION SMOOTHING: 
#### Best lambda value trigram_lambda = 0.7000000000000001 bigram_lambda = 0.05 unigram_lambda = 0.24999999999999994

In [25]:
# Greedy search
from default import *
from zhsegment_check import fscore

Pw_bigram = Pdist_bigram(data=datafile("data/count_2w.txt"))
Pw_trigram = Pdist_trigram(data=datafile("data/count_3w.txt"))
Pw_unigram = Pdist_unigram(data=datafile("data/count_1w.txt"), missingfn=avoid_long_words)

interp_lambdas3 = [i for i in range(1,99)]

best_f1 = 0
best_lambda = 0

for interp_lambda3 in interp_lambdas3:   
    interp_lambdas2 = [i for i in range(1,99-interp_lambda3-1)]
    interp_lambda3 *= 0.01
    for interp_lambda2 in interp_lambdas2: 
        interp_lambda2 *= 0.01
    
        segmenter = Segment(Pw_unigram, Pw_bigram, Pw_trigram, n_gram=3, smooth_prob=True, interp_lambda3=interp_lambda3, interp_lambda2=interp_lambda2)
        output_full = []
        with open("data/input/dev.txt") as f:
            for line in f:
                output = " ".join(segmenter.segment(line.strip()))
                output_full.append(output)

        with open('data/reference/dev.out', 'r') as refh:
            ref_data = [str(x).strip() for x in refh.read().splitlines()]
            tally = fscore(ref_data, output_full)
            if best_f1 < tally:
                best_f1 = tally
                best_interp_lambda3 = interp_lambda3
                best_interp_lambda2 = interp_lambda2
                best_interp_lambda1 = 1 - interp_lambda3 - interp_lambda2
                print("BEST: ", best_f1, " ", best_interp_lambda3, " ", best_interp_lambda2, " ", best_interp_lambda1)

BEST:  0.9245595109672778   0.01   0.01   0.98
BEST:  0.9246861924686193   0.01   0.72   0.27
BEST:  0.9247870651075503   0.01   0.74   0.25
BEST:  0.9249205890846087   0.01   0.76   0.22999999999999998
BEST:  0.9250649725671384   0.29   0.46   0.24999999999999994
BEST:  0.9251985559566788   0.31   0.46   0.22999999999999993
BEST:  0.9252437703141929   0.6   0.15   0.25
BEST:  0.9253774470851694   0.62   0.15   0.23
BEST:  0.9255088783022953   0.68   0.05   0.26999999999999996
BEST:  0.9256102845587173   0.7000000000000001   0.05   0.24999999999999994


### Printing out sentences that are segmented falsely for Bigram model with jm-smoothing

In [26]:
from default import *

Pw_unigram = Pdist_unigram(data=datafile("data/count_1w.txt"), missingfn=avoid_long_words)
Pw_bigram = Pdist_bigram(data=datafile("data/count_2w.txt"))
Pw_trigram = Pdist_trigram(data=datafile("data/count_3w.txt"))
segmenter = Segment(Pw_unigram, Pw_bigram, Pw_trigram, n_gram=2, smooth_prob=True) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)

from zhsegment_check import fscore
with open('data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    
    for itr, ref in enumerate(ref_data):
        if ref != output_full[itr]:
            print("Pred: ", output_full[itr])
            print("Gt  :", ref, "\n")
    tally = fscore(ref_data, output_full)


Pred:  这 三 个 项目 是 分别 由 国务院 发展 研究 中心 国际 技术 经济 研究所 上海 分 所 和 上海市 浦东 继续 教育 中心 ， 与 美国 知识 信息 网络 公司 、 世界 学习 组织 、 海 赛克 公司 签订 的 。
Gt  : 这 三 个 项目 是 分别 由 国务院 发展 研究 中心 国际 技术 经济 研究所 上海 分所 和 上海市 浦东 继续 教育 中心 ， 与 美国 知识 信息 网络 公司 、 世界 学习 组织 、 海赛克 公司 签订 的 。 

Pred:  中国 将 于 十二月 举办 “ 华夏艺术节 ”
Gt  : 中国 将 于 十二月 举办 “ 华夏 艺术节 ” 

Pred:  新华社 北京 九月 一日 电 （ 记者 高 建 新 ）
Gt  : 新华社 北京 九月 一日 电 （ 记者 高建新 ） 

Pred:  为 加强 海内外 炎黄子孙 的 文化 交流 和 联系 ， 中国 将 于 今年 十二月 八日 到 十八日 在 北京 和 深圳 两 地 同时 举办 “ 华夏艺术节 ” 。
Gt  : 为 加强 海内外 炎黄子孙 的 文化 交流 和 联系 ， 中国 将 于 今年 十二月 八日 到 十八日 在 北京 和 深圳 两 地 同时 举办 “ 华夏 艺术节 ” 。 

Pred:  其 宗旨 是 ， 以 中华 民族 传统 文化 为 纽带 ， 联系 和 团结 海内外 同胞 ， 弘扬 民族 文化 ， 展示 祖国 优秀 艺术 的 魅力 ， 展示 华侨 、 华人 艺术 家 在 艺术 领域 和 世界 文化 交流 中 的 成就 和 风采 。
Gt  : 其 宗旨 是 ， 以 中华 民族 传统 文化 为 纽带 ， 联系 和 团结 海内外 同胞 ， 弘扬 民族 文化 ， 展示 祖国 优秀 艺术 的 魅力 ， 展示 华侨 、 华人 艺术家 在 艺术 领域 和 世界 文化 交流 中 的 成就 和 风采 。 

Pred:  艺术 节 的 内容 主要 包括 隆重 的 开幕式 和 闭幕 式 ， 各 具 特色 的 大型 文艺表演 以及 美术、摄影 展览 。
Gt  : 艺术节 的 内容 主要 包括 隆重 的 开幕式 和 闭幕式 ， 各 具 特色 的 大型 文艺 表演 以及 美术 、 摄影 展览 。 

Pred:  据 介绍 ， 十二月 八日 在 深圳 体育馆 